**Installing & Importing Required Libraries**

In [8]:
!pip install transformers

In [9]:
import re
import json
import h5py
import tqdm
import numpy as np
import pandas as pd
import joblib as jb
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split

**Mounting Google Drive for importing the Data Files which will be used in the Tokenization**

**Importing Updated Recipe json File which contains Recipe Data**

In [10]:
df_new = jb.load('/home/dev/Dataset_Project/data_v1.pickle')

**Converting Recipe json File into a DataFrame**

In [11]:
df = pd.DataFrame(df_new)

In [12]:
df.head()

,ID,title,ingredients,ingredient_phrase,continent,region,sub_region,instructions,cuisine
0,2610,Egyptian Lentil Soup,"[black pepper, red lentil, coriander, onion, c...","[3 cups water, 1 cup red lentils, 1 roma tomat...",African,Middle Eastern,Egyptian,"[Place 3 cups water, lentils, tomato, carrot, ...",Middle Eastern
1,2611,Egyptian Green Beans with Carrots,"[tomato paste, cardamom, bay leaf, chicken sto...","[1 tablespoon vegetable oil, 1 large onion , d...",African,Middle Eastern,Egyptian,"[Heat oil in a pot over medium heat., Cook and...",Middle Eastern
2,2612,Egyptian Bamia,"[tomato sauce, lamb shoulder, okra, onion, oli...","[1/4 cup olive oil, 1 large onion , finely cho...",African,Middle Eastern,Egyptian,[Heat olive oil in a large saucepan over mediu...,Middle Eastern
3,2613,Magpie's Easy Falafel Cakes,"[coriander, cornmeal, garbanzo bean, sauce, cr...","[1/2 small onion , minced, 1 1/2 teaspoons oli...",African,Middle Eastern,Egyptian,[Cook the onions in 1 1/2 teaspoons of olive o...,Middle Eastern
4,2614,Dukkah,"[black pepper, hazelnut, coriander seed, sea s...","[2/3 cup hazelnuts, 1/2 cup sesame seeds, 2 ta...",African,Middle Eastern,Egyptian,[Preheat the oven to 350 degrees f. Place the ...,Middle Eastern


**Formatting the Instructions of the Recipe by Performing operations like removing '\t' from the beginning of the instructions, inserting ';' at the end of each instruction, etc**

In [13]:
list_of_instrns = []
for row in range(0,len(df)):
    instr = df.iloc[row]['instructions']

    strg = ""
    length = len(instr) - 1
    count = 0
    for instruction in instr:
        processed_instr = []
        for j in range(0,len(instruction)):
            if(instruction[j]=='|' or instruction[j]=='\t'):
                continue
            elif(instruction[j]==' '):
                if(instruction[j-1]!='|'):
                   strg = strg + instruction[j]
            elif(instruction[j] == '.') and (j!=len(instruction)-1) and (instruction[j-1].isdigit()==False):
                strg = strg + ' '
                strg = strg + instruction[j]
            elif(instruction[j]>='a' and instruction[j]<='z') or (instruction[j]>='A' and instruction[j]<='Z') :
                 strg =  strg + instruction[j].lower()
            elif(instruction[j] == ','):
                  strg =  strg + ' '
                  strg =  strg + ','
            elif(instruction[j].isdigit()):
                if(instruction[j+1] == '.') or (instruction[j+2] == '.'):
                    continue
                else:
                    strg = strg + instruction[j]

        if(count!=length):
            strg = strg + ' '
            strg = strg + ';'
            strg = strg + ' '


        count = count + 1

    processed_instr.append(strg)
    list_of_instrns.append(processed_instr)

**Deleting the current "instructions" column from the DataFrame and inserting the modified Instructions by Creating the new "instructions" column**

In [14]:
df.drop('instructions', inplace=True, axis=1)

In [15]:
df['instructions'] = list_of_instrns

In [16]:
df.head()

,ID,title,ingredients,ingredient_phrase,continent,region,sub_region,cuisine,instructions
0,2610,Egyptian Lentil Soup,"[black pepper, red lentil, coriander, onion, c...","[3 cups water, 1 cup red lentils, 1 roma tomat...",African,Middle Eastern,Egyptian,Middle Eastern,"[place 3 cups water , lentils , tomato , carro..."
1,2611,Egyptian Green Beans with Carrots,"[tomato paste, cardamom, bay leaf, chicken sto...","[1 tablespoon vegetable oil, 1 large onion , d...",African,Middle Eastern,Egyptian,Middle Eastern,[heat oil in a pot over medium heat ; cook and...
2,2612,Egyptian Bamia,"[tomato sauce, lamb shoulder, okra, onion, oli...","[1/4 cup olive oil, 1 large onion , finely cho...",African,Middle Eastern,Egyptian,Middle Eastern,[heat olive oil in a large saucepan over mediu...
3,2613,Magpie's Easy Falafel Cakes,"[coriander, cornmeal, garbanzo bean, sauce, cr...","[1/2 small onion , minced, 1 1/2 teaspoons oli...",African,Middle Eastern,Egyptian,Middle Eastern,[cook the onions in 1 12 teaspoons of olive oi...
4,2614,Dukkah,"[black pepper, hazelnut, coriander seed, sea s...","[2/3 cup hazelnuts, 1/2 cup sesame seeds, 2 ta...",African,Middle Eastern,Egyptian,Middle Eastern,[preheat the oven to 350 degrees f . place the...


**Dividing whole Data into Train and Test part with the Ratio of Train to Test is 0.96 : 0.04**

In [17]:
df.drop('region', axis=1, inplace=True)

In [18]:
df.head()

,ID,title,ingredients,ingredient_phrase,continent,sub_region,cuisine,instructions
0,2610,Egyptian Lentil Soup,"[black pepper, red lentil, coriander, onion, c...","[3 cups water, 1 cup red lentils, 1 roma tomat...",African,Egyptian,Middle Eastern,"[place 3 cups water , lentils , tomato , carro..."
1,2611,Egyptian Green Beans with Carrots,"[tomato paste, cardamom, bay leaf, chicken sto...","[1 tablespoon vegetable oil, 1 large onion , d...",African,Egyptian,Middle Eastern,[heat oil in a pot over medium heat ; cook and...
2,2612,Egyptian Bamia,"[tomato sauce, lamb shoulder, okra, onion, oli...","[1/4 cup olive oil, 1 large onion , finely cho...",African,Egyptian,Middle Eastern,[heat olive oil in a large saucepan over mediu...
3,2613,Magpie's Easy Falafel Cakes,"[coriander, cornmeal, garbanzo bean, sauce, cr...","[1/2 small onion , minced, 1 1/2 teaspoons oli...",African,Egyptian,Middle Eastern,[cook the onions in 1 12 teaspoons of olive oi...
4,2614,Dukkah,"[black pepper, hazelnut, coriander seed, sea s...","[2/3 cup hazelnuts, 1/2 cup sesame seeds, 2 ta...",African,Egyptian,Middle Eastern,[preheat the oven to 350 degrees f . place the...


In [19]:
train,test = train_test_split(df, train_size=0.96, random_state= 2)

**Displaying the Size of Train and Test Part and Resetting to the Default Index of these portions**

In [20]:
print("Train Portion size is: ",train.shape)
print("Test Portion size is: ",test.shape)

Train Portion size is:  (113359, 8)
Test Portion size is:  (4724, 8)


In [21]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

**Defining the function that will be used for Converting the Dataset into Text Data Format so that the the Data can be Tokenize**

In [22]:
def df_to_plaintext_file(input_df, output_file):
    print("Writing to", output_file)
    with open(output_file, 'w', encoding="utf-8") as f:
        for index, row in input_df.iterrows():
            # Skip the recipes that are not in the top 5 cuisines
            # Add the cuisine token for top 5 cuisines
            #cuisine_token = f"<CUISINE_{row['cuisine'].upper()}>"
            title = row.title
            instructions = ' '.join(row.instructions).split('.')[:-1]  # Assuming instructions are in a list
            ingredients = row.ingredient_phrase
            keyword = row.ingredients

            # For debugging every 40000 entries
            if index % 40000 == 0:
                print(index)
                print("ingreds --->", ingredients)
                print("keywords --->", keyword)

            # Construct the formatted text
            res = f"<RECIPE_START> <INPUT_START> " + " <NEXT_INPUT> ".join(keyword) + \
                  " <INPUT_END> <TITLE_START> " + title + "<TITLE_END> <INGR_START> " + \
                  " <NEXT_INGR> ".join(ingredients) + " <INGR_END> <INSTR_START> " + \
                  " <NEXT_INSTR> ".join(instructions) + " <INSTR_END> <RECIPE_END>"

            f.write(f"{res}\n")

**Saving the Processed Train and Test Files to the Custom Path**

In [23]:
df_to_plaintext_file(train, '/home/Dataset_Project/train_temp.txt')
df_to_plaintext_file(test, '/home/Dataset_Project/test_temp.txt')

Writing to /home/hiren20066/BTP_Dev/Dataset_Project/train_temp.txt
0
ingreds ---> ['1/3 cup unsalted pistachios', '1 tablespoon fresh thyme , finely chopped', '1 1/2 teaspoons fresh rosemary , finely chopped', '1/3 cup extra virgin olive oil', 'salt & freshly ground black pepper', '1 1/2 lbs racks of lamb , frenched', '6 ounces thinly sliced pancetta', '8 scallions , sliced']
keywords ---> ['rosemary', 'pancetta', 'thyme', 'extra virgin olive oil', 'salt black pepper', 'pistachio', 'scallion', 'rack lamb']
40000
ingreds ---> ['500 g basmati rice or 500 g long-grain rice', '1 kg herbs ( parsley , dill , coriander , fenugreek , spring onion ends )', 'cooking oil', 'salt', 'black pepper']
keywords ---> ['black pepper', 'basmati rice', 'cooking oil', 'salt', 'herb']
80000
ingreds ---> ['4 cups Mexican blend cheese , shredded ( or Monterey Jack )', '1 cup beer', '4 ounces canned diced green chiles', '3 -4 garlic cloves or 1 tablespoon garlic powder', '1 tablespoon fresh cilantro , chopped',

**Initializing the GPT2 Tokenizer and Adding special Tokens defined by us to Define the different parts of the Recipe like its title, constituting ingredeints, etc**

In [24]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", do_lower_case=False)
# tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
special_tokens = {
    "additional_special_tokens": [
        '<RECIPE_START>',
        '<INPUT_START>',
        '<NEXT_INPUT>',
        '<INPUT_END>',
        '<INGR_START>',
        '<NEXT_INGR>',
        '<INGR_END>',
        '<INSTR_START>',
        '<NEXT_INSTR>',
        '<INSTR_END>',
        '<TITLE_START>',
        '<TITLE_END>',
        '<RECIPE_END>'
    ]
}
tokenizer.add_special_tokens(special_tokens)

end_token_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]

hf = h5py.File("/home/Dataset_Project/data_temp.h5", "w")
for filename in ["/home/Dataset_Project/test_temp", "/home/Dataset_Project/train_temp"]:
    out_np = []
    data = open(filename+".txt", "r")
    num = 0
    rows = 0
    last=[]

    # Determine if we are processing the training or test set
    is_train = "train" in filename

    # Get the corresponding part of the DataFrame
    current_df = train if is_train else test

    for line, (_, row) in zip(data, current_df.iterrows()):
        num += 1
        if num % 10000 == 0:
            print("Read "+str(num)+" Written: "+str(rows))
        text_tokens = tokenizer.tokenize(line)
        if len(text_tokens) > 1024:
            continue
        text_tokens_ids = tokenizer.convert_tokens_to_ids(text_tokens)

        if (len(last) + len(text_tokens_ids)) <= 1024:
            last+=text_tokens_ids
        else:
            while len(last) < 1024:
                last.append(end_token_id)
            out_np.append(last)
            last=text_tokens_ids
            rows+=1
    out_mat = np.matrix(out_np)
    print(out_mat.shape)
    hf.create_dataset(filename, data=out_mat)
hf.close()

(1679, 1024)
Read 10000 Written: 3477
Read 20000 Written: 7005
Read 30000 Written: 10537
Read 40000 Written: 14071
Read 50000 Written: 17580
Read 60000 Written: 21041
Read 70000 Written: 24556
Read 80000 Written: 28098
Read 90000 Written: 31625
Read 100000 Written: 35132
Read 110000 Written: 38612
(39774, 1024)


**Displaying the Final Length of Tokenizer**

In [25]:
len(tokenizer)

50270

**Displaying the Final Number of Recipes Downsampled**

In [26]:
t = []
with open('/home/Dataset_Project/train_temp.txt') as file1:
    for f in file1:
        t.append(f)

In [27]:
print('No of recipes downsampled for prototyping: ',len(t))

No of recipes downsampled for prototyping:  113359
